<a href="https://www.kaggle.com/code/pes1ug22am164/image-part?scriptVersionId=171996997" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# uncomment on google colab, comment on kaggle.

In [1]:
!mkdir ~/.kaggle
from google.colab import files
files.upload()
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d shusrith/betterest-preproc

Saving kaggle.json to kaggle.json
 28% 10.0M/35.1M [00:00<00:00, 104MB/s]
100% 35.1M/35.1M [00:00<00:00, 202MB/s]


In [ ]:
!unzip /content/betterest-preproc.zip

In [ ]:
# !kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images

In [ ]:
# !unzip /content/alzheimers-dataset-4-class-of-images.zip

# PRE PROCESS
.JPG TO .NPY

In [ ]:
# import os
# import cv2
# import numpy as np

# def jpg_to_npy_in_folders(input_folder, output_folder):
#     os.makedirs(output_folder, exist_ok=True)
#     for i, item in enumerate(os.listdir(input_folder)):
#         item_path = os.path.join(input_folder, item)
#         if os.path.isdir(item_path):
#             jpg_to_npy_in_folders(item_path, output_folder)
#         elif os.path.isfile(item_path) and item.endswith(".jpg"):
#             img = cv2.imread(item_path)
#             img_array = np.array(img)
#             output_file_path = os.path.join(output_folder, os.path.splitext(item)[0] + '.npy')
#             np.save(output_file_path, img_array)

# # input_folder_path = "/content/Alzheimer_s Dataset/train"
# input_folder_path = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
# output_folder_path = "/kaggle/working/train"

# jpg_to_npy_in_folders(input_folder_path, output_folder_path)



In [ ]:
# import os
# import cv2
# import numpy as np

# def jpg_to_npy_in_folders(input_folder, output_folder):
#     os.makedirs(output_folder, exist_ok=True)
#     for i, item in enumerate(os.listdir(input_folder)):
#         item_path = os.path.join(input_folder, item)
#         if os.path.isdir(item_path):
#             jpg_to_npy_in_folders(item_path, output_folder)
#         elif os.path.isfile(item_path) and item.endswith(".jpg"):
#             img = cv2.imread(item_path)
#             img_array = np.array(img)
#             folder = input_folder.split("/")[-1]
#             output_file_path = os.path.join(output_folder, f"{folder}{i}.npy")
#             np.save(output_file_path, img_array)

# # input_folder_path = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
# # output_folder_path = "/kaggle/working/train"

# # jpg_to_npy_in_folders(input_folder_path, output_folder_path)
# jpg_to_npy_in_folders("/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test", "/kaggle/working/test")
# # jpg_to_npy_in_folders("/content/Alzheimer_s Dataset/test", "/kaggle/working/test")



In [ ]:
# import os
# import numpy as np
# directory = '/kaggle/working/train'

# files = [file for file in os.listdir(directory) if file.endswith('.npy')]
# len(files)


In [ ]:
# mildDem = [i for i in files if i.startswith("mildDem")]
# moderateDem = [i for i in files if "moderateDem" in i]
# nonDem = [i for i in files if "nonDem" in i]
# veryMild = [i for i in files if "verymildDem" in i]
# print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

In [ ]:
# import imgaug.augmenters as iaa

# # Define the augmenter
# augmenter = iaa.Sequential([
#     iaa.Affine(rotate=(-10, 10)),  # Rotate the image by -10 to 10 degrees
#     iaa.GaussianBlur(sigma=(0.0, 1.0)),  # Apply Gaussian blur with a sigma ranging from 0.0 to 1.0
#     iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add Gaussian noise with a scale of 0 to 0.05 times 255
#     iaa.Multiply((0.9, 1.1)),
#     iaa.LinearContrast((0.9, 1.1)),
# ])

# augmented_images = []

# for path in moderateDem:
#     path = os.path.join("/kaggle/working/train", path)
#     image = np.load(path)

#     for _ in range(35):
#         augmented_image = augmenter.augment_image(image)
#         augmented_images.append(augmented_image)
# augmented_images = np.array(augmented_images)

In [ ]:
# len(augmented_images)

In [ ]:
# import cv2
# import os


# output_directory = "/kaggle/working/train"

# os.makedirs(output_directory, exist_ok=True)

# for i, image in enumerate(augmented_images):
#     output_path = os.path.join(output_directory, f"moderateDem{i+53}.npy")
#     np.save(output_path, image)


# print("Augmented images saved successfully.")


In [ ]:
# augmented_images = []

# for path in mildDem:
#     path = os.path.join("/kaggle/working/train", path)
#     image = np.load(path)

#     for _ in range(2):
#         augmented_image = augmenter.augment_image(image)
#         augmented_images.append(augmented_image)
# augmented_images = np.array(augmented_images)

In [ ]:
# len(augmented_images)

In [ ]:
# output_directory = "/kaggle/working/train"

# os.makedirs(output_directory, exist_ok=True)

# for i, image in enumerate(augmented_images):
#     output_path = os.path.join(output_directory, f"mildDem{i+718}.npy")
#     np.save(output_path, image)


# print("Augmented images saved successfully.")

In [ ]:
# import os
# import numpy as np
# directory = '/kaggle/working/train'

# files = [file for file in os.listdir(directory) if file.endswith('.npy')]
# len(files)

In [ ]:
# mildDem = [i for i in files if i.startswith("mildDem")]
# moderateDem = [i for i in files if "moderateDem" in i]
# nonDem = [i for i in files if "nonDem" in i]
# veryMild = [i for i in files if "verymildDem" in i]
# print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

In [ ]:
# import pandas as pd
# l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
# x = []
# for i in l:
#     for j in i[0]:
#         path = os.path.join("/kaggle/working/train", j)
#         x.append([i[1], path])

# df = pd.DataFrame(x, columns=["Class", "Path"])
# x = df["Class"].value_counts()
# x

In [ ]:
# images = []
# for i in df["Path"]:
#     images.append(np.load(i))
# images = np.array(images)

In [ ]:
# from imblearn.over_sampling import SMOTE

# sm = SMOTE(random_state=42)
# flattened_images = [image.flatten() for image in images]
# image_shape = images[0].shape
# X_res, y_res = sm.fit_resample(flattened_images, np.array(df["Class"]))  # `labels` should be your list of class labels


In [ ]:
# res_images = [np.array(x).reshape(image_shape) for x in X_res]

In [ ]:
# !rm -rf /kaggle/working/train

In [ ]:
# import os

# names = {0:"milDem", 1:"moderateDim", 2:"nonDem", 3:"veryMildDem"}

# # Create the directory if it doesn't exist
# os.makedirs("/kaggle/working/train/", exist_ok=True)

# for i in range(len(y_res)):
#     np.save(f"/kaggle/working/train/{names[y_res[i]]}{i}.npy", res_images[i])

In [ ]:
# import pandas as pd
# labels = pd.DataFrame(y_res)
# labels.value_counts()

In [ ]:
# images = np.array(res_images)

In [4]:
import os
import numpy as np
directory = '/content/train'

files = [file for file in os.listdir(directory) if file.endswith('.jpg')]
len(files)

10240

In [5]:
mildDem = [i for i in files if i.startswith("milDem")]
moderateDem = [i for i in files if "moderateDim" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "veryMildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

2560 2560 2560 2560


In [6]:
import pandas as pd
l = [[mildDem, "0"], [moderateDem, "1"], [nonDem, "2"], [veryMild, "3"]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/content/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
x = df["Class"].value_counts()
x

Class
0    2560
1    2560
2    2560
3    2560
Name: count, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(df["Path"], df["Class"], test_size=0.2, random_state=42)

In [15]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

train_df = pd.DataFrame({
    'filename': train_images,
    'class': train_labels
})
val_df = pd.DataFrame({
    'filename': test_images,
    'class': test_labels
})

datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='filename',
    y_col='class',
    target_size=(130, 110),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=None,
    x_col='filename',
    y_col='class',
    target_size=(130, 110),
    batch_size=32,
    class_mode='categorical'
)

Found 8192 validated image filenames belonging to 4 classes.
Found 2048 validated image filenames belonging to 4 classes.


In [ ]:
# images = []
# for i in df["Path"]:
#     images.append(np.load(i))
# images = np.array(images)

In [ ]:
# from keras.utils import to_categorical

# df["Class"] = to_categorical(df["Class"], num_classes=4)


In [11]:
# from keras.preprocessing.image import ImageDataGenerator
# from sklearn.model_selection import train_test_split

# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# datagen = ImageDataGenerator(rescale=1.0/255.0)

# input_shape = (208, 176, 3)

# train_generator = datagen.flow_from_dataframe(
#     dataframe=train_df,
#     directory=None,
#     x_col="Path",
#     y_col="Class",
#     class_mode="categorical",
#     target_size=(input_shape[0], input_shape[1]),
#     batch_size=32
# )



In [12]:
# test_generator = datagen.flow_from_dataframe(
#     dataframe=test_df,
#     directory=None,
#     x_col="Path",
#     y_col="Class",
#     class_mode="categorical",
#     target_size=(input_shape[0], input_shape[1]),
#     batch_size=32
# )

In [ ]:
# import gc

# # Assume `arr` is your numpy array
# del images

# # Run the garbage collector
# gc.collect()

0

In [8]:
# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# train_labels_encoded = label_encoder.fit_transform(train_labels)
# test_labels_encoded = label_encoder.transform(test_labels)

# # Verify the encoded labels
# print(train_labels_encoded)
# print(test_labels_encoded)

In [9]:
# print(train_images.shape, test_images.shape, train_labels_encoded.shape, test_labels_encoded.shape)

In [10]:
# print(test_labels)
# test_labels_encoded

In [ ]:
# print(x.index, x.values, x.array)

In [ ]:
# s = x / len(files)
# class_weights = {i:j for i, j in zip(s.index, s.values)}

# class_weights
# Optional normalization (weights sum to 1)
# total_weight = sum(class_weights.values())
# class_weights = {class_label: weight / total_weight for class_label, weight in class_weights.items()}


In [16]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
import numpy as np

def conv_block(filters, act='relu'):
    """Defining a Convolutional NN block for a Sequential CNN model. """

    block = Sequential()
    block.add(Conv2D(filters, 3, activation=act, padding='same'))
    block.add(Conv2D(filters, 3, activation=act, padding='same'))
    block.add(BatchNormalization())
    block.add(MaxPooling2D((2, 2)))

    return block

In [17]:
def dense_block(units, dropout_rate, act='relu'):
    """Defining a Dense NN block for a Sequential CNN model. """

    block = Sequential()
    block.add(Dense(units, activation=act))
    block.add(BatchNormalization())
    block.add(Dropout(dropout_rate))

    return block

In [ ]:
# !pip install tensorflow_addons


In [18]:
import tensorflow as tf

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes, **kwargs):
        super(F1Score, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_state(self):
        self.precision.reset_states()
        self.recall.reset_states()

METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'),
           tf.keras.metrics.AUC(name='auc'),
           F1Score(num_classes=4)]

In [21]:
def construct_model(act='relu'):
    """Constructing a Sequential CNN architecture for performing the classification task. """

    model = Sequential([
        Input(shape=(130, 110, 3)),
#         conv_block(16),
        conv_block(32),
        conv_block(64),
        conv_block(128),
        Dropout(0.2),
        conv_block(256),
        Dropout(0.2),
        Flatten(),
        dense_block(512, 0.7),
        dense_block(256, 0.5),
        dense_block(64, 0.3),
        Dense(4, activation='softmax')
    ], name = "cnn_model")

    return model

In [22]:
model = construct_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=METRICS)

model.summary()


Model: "cnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_7 (Sequential)   (None, 65, 55, 32)        10272     
                                                                 
 sequential_8 (Sequential)   (None, 32, 27, 64)        55680     
                                                                 
 sequential_9 (Sequential)   (None, 16, 13, 128)       221952    
                                                                 
 dropout_5 (Dropout)         (None, 16, 13, 128)       0         
                                                                 
 sequential_10 (Sequential)  (None, 8, 6, 256)         886272    
                                                                 
 dropout_6 (Dropout)         (None, 8, 6, 256)         0         
                                                                 
 flatten_1 (Flatten)         (None, 12288)             0 

In [23]:
from keras.optimizers import Adam
def lr_schedule(epoch, learning_rate):
    if epoch < 20:
        return learning_rate
    else:
        if epoch % 2 == 0:
          return learning_rate * 0.95
        else:
          return learning_rate

lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)

initial_learning_rate = 0.001
optimizer = Adam(learning_rate=initial_learning_rate)

In [24]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_acc') > 0.995:
            print("\nReached accuracy threshold! Terminating training.")
            self.model.stop_training = True

my_callback = MyCallback()

CALLBACKS = [my_callback, lr_scheduler]

In [25]:
EPOCHS = 50

history = model.fit(train_generator, validation_data=(validation_generator), callbacks=CALLBACKS, epochs=EPOCHS)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
256/256 [==============================] - ETA: 0s - loss: 1.4000 - acc: 0.4315 - auc: 0.6933 - f1_score: 0.3755

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:2723: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


256/256 [==============================] - 36s 77ms/step - loss: 1.4000 - acc: 0.4315 - auc: 0.6933 - f1_score: 0.3755 - val_loss: 6.9080 - val_acc: 0.2573 - val_auc: 0.5049 - val_f1_score: 0.2573 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50
256/256 [==============================] - 20s 77ms/step - loss: 0.8207 - acc: 0.6295 - auc: 0.8776 - f1_score: 0.5964 - val_loss: 13.9957 - val_acc: 0.2573 - val_auc: 0.5093 - val_f1_score: 0.2573 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
256/256 [==============================] - 24s 93ms/step - loss: 0.7027 - acc: 0.6766 - auc: 0.9086 - f1_score: 0.6525 - val_loss: 3.2876 - val_acc: 0.2896 - val_auc: 0.6221 - val_f1_score: 0.2786 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/50
256/256 [==============================] - 23s 89ms/step - loss: 0.6092 - acc: 0.7311 - auc: 0.9

In [ ]:


# # Assuming train_images and test_images are your input image data


# model = Sequential()
# model.add(Input(shape=input_shape))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4, activation='softmax'))


In [ ]:
# import math
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# history = model.fit(train_images, train_labels_encoded,
#                     steps_per_epoch = int(len(train_images) / 32),
#                     epochs=50,
#                     validation_data=(test_images, test_labels_encoded),
#                     sample_weight=sample_weights,
#                     callbacks=[lr_scheduler])

In [ ]:
# import tensorflow as tf
# import numpy as np
# from sklearn.metrics import roc_auc_score

# y_prob = model.predict(test_images)
# y_prob = tf.nn.softmax(y_prob).numpy()

# # Compute ROC AUC for each class
# roc_auc = roc_auc_score(test_labels_encoded, y_prob, average='macro')

# print("ROC AUC:", roc_auc)

 5/64 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step

W0000 00:00:1713409672.384326      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


64/64 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step
ROC AUC: 0.9994468326560391


In [28]:
import os
import numpy as np
directory = '/content/test'

files = [file for file in os.listdir(directory) if file.endswith('.jpg')]
len(files)

1279

In [29]:
mildDem = [i for i in files if i.startswith("MildDem")]
moderateDem = [i for i in files if "ModerateDem" in i]
nonDem = [i for i in files if "NonDem" in i]
veryMild = [i for i in files if "VeryMildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

179 12 640 448


In [31]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/content/test", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
df["Class"].value_counts()


Class
2    640
3    448
0    179
1     12
Name: count, dtype: int64

In [39]:
from keras.preprocessing.image import load_img, img_to_array
img = [load_img(i, target_size=(130, 110)) for i in df["Path"]]
img = [img_to_array(im) for im in img]
img = [im / 255.0 for im in img]
img = np.stack(img, axis=0)


In [35]:
labels = df["Class"]
labels = to_categorical(labels, num_classes=4)
print(labels.shape)

(1279, 4)


In [36]:
l = []
for i in labels:
  for j in range(4):
    if i[j] == 1:
      l.append(j)


In [40]:
y_prob = model.predict(img)
y_prob = np.argmax(y_prob, axis=-1)

40/40 [==============================] - 3s 60ms/step


In [41]:
from sklearn.metrics import classification_report

report = classification_report(l, y_prob, output_dict=True)
for i in report:
  print(i, report[i])

0 {'precision': 0.5471698113207547, 'recall': 0.6480446927374302, 'f1-score': 0.5933503836317136, 'support': 179}
1 {'precision': 1.0, 'recall': 0.3333333333333333, 'f1-score': 0.5, 'support': 12}
2 {'precision': 0.7824620573355818, 'recall': 0.725, 'f1-score': 0.7526358475263585, 'support': 640}
3 {'precision': 0.6404255319148936, 'recall': 0.671875, 'f1-score': 0.6557734204793029, 'support': 448}
accuracy 0.6919468334636435
macro avg {'precision': 0.7425143501428075, 'recall': 0.5945632565176908, 'f1-score': 0.6254399129093438, 'support': 1279}
weighted avg {'precision': 0.7018215412189678, 'recall': 0.6919468334636435, 'f1-score': 0.6940446860529116, 'support': 1279}


In [42]:
model.evaluate(img, labels)

 5/40 [==>...........................] - ETA: 1s - loss: 1.5557 - acc: 0.6687 - auc: 0.8514 - f1_score: 0.6708

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:2723: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


40/40 [==============================] - 1s 23ms/step - loss: 1.3500 - acc: 0.6919 - auc: 0.8760 - f1_score: 0.6920


[1.349985957145691, 0.691946804523468, 0.8760294914245605, 0.6920062303543091]